# Q2. (40 points)
I am also sharing with you, John McDonnell's python implementation of Anderson's Rational Model of Categorization (rational.py).
Modify the code to obtain category predictions for the data I have shared with you. 

In [7]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [8]:
X = pd.read_csv('X.csv', header=None).values
y_test = pd.read_csv('y.csv', header=None).values

# Separate data into weight, height, and labels
weights = X[:, 0]
heights = X[:, 1]

# Categories: 1 = small, 2 = average, 3 = large
labels = X[:, 2]

# Split data by category
small = X[labels == 1]
average = X[labels == 2]
large = X[labels == 3]

In [9]:
# Calculating mean and standard deviation of weight and height for each category.
params = {
    'small': {
        'weight_mean': np.mean(small[:, 0]),
        'weight_std': np.std(small[:, 0]),
        'height_mean': np.mean(small[:, 1]),
        'height_std': np.std(small[:, 1])
    },
    'average': {
        'weight_mean': np.mean(average[:, 0]),
        'weight_std': np.std(average[:, 0]),
        'height_mean': np.mean(average[:, 1]),
        'height_std': np.std(average[:, 1])
    },
    'large': {
        'weight_mean': np.mean(large[:, 0]),
        'weight_std': np.std(large[:, 0]),
        'height_mean': np.mean(large[:, 1]),
        'height_std': np.std(large[:, 1])
    }
}

# Incorporating posterior formula:
# P(cN=j|xN,xN-1,cN-1) = [p(xN|cN=j,xN-1,cN-1) . p(cN=j|cN-1)] / ∑c[p(xN|cN=j,xN-1,cN-1) . p(cN=j|cN-1)]

In [10]:
def likelihood(weight, height, category, params):
    # Calculating likelihood for weight and height
    weight_likelihood = norm.pdf(weight, params[category]['weight_mean'], params[category]['weight_std'])
    height_likelihood = norm.pdf(height, params[category]['height_mean'], params[category]['height_std'])
    
    # Adjusting likelihood based on weight's higher importance
    likelihood = (weight_likelihood ** 0.7) * (height_likelihood ** 0.3)
    return likelihood

def posterior_prob(weight, height, params):
    # Calculating posterior probability for each category
    posterior_probs = {}
    categories = ['small', 'average', 'large']
    cat_labels = {category: i + 1 for i, category in enumerate(categories)}
    
    for category in categories:
        cat_label = cat_labels[category]
        posterior_probs[cat_label] = likelihood(weight, height, category, params)
    
    # Normalizing the probabilities so that they sum to 1
    total_posterior = sum(posterior_probs.values())
    for cat_label in posterior_probs:
        posterior_probs[cat_label] /= total_posterior
    
    return posterior_probs

def predict_cat_label(posterior_probs):
    # Returning the category with the highest posterior probability
    return max(posterior_probs, key=posterior_probs.get)

In [11]:
def evaluate_model(X, labels, params):
    # Predict labels for X.csv and compare with given labels
    predictions = []
    accuracy = 0
    
    for i in range(len(X)):
        weight, height = X[i, 0], X[i, 1]
        true_label = X[i, 2]
        
        # Calculate posterior probabilities for this instance 
        posterior_probs = posterior_prob(weight, height, params)
        
        # Find the category with the highest posterior probability
        predicted_label = predict_cat_label(posterior_probs)
        
        # Append the prediction
        predictions.append(predicted_label)
        
        print(f"Sample {i + 1}\nTrue label = {true_label}, Predicted label = {predicted_label}, Posterior probs = {posterior_probs}\n")
        
        # Check if prediction matches true label
        if predicted_label == true_label:
            accuracy += 1
    
    accuracy /= len(labels)
    return predictions, accuracy

In [12]:
def predict_test_instances(y_test, params):
    # Predicting labels for y.csv
    y_predictions = []
    
    for i, test_instance in enumerate(y_test):
        weight, height = test_instance
        posterior_probs = posterior_prob(weight, height, params)
        
        # Finding the category with the highest posterior probability
        predicted_label = predict_cat_label(posterior_probs)
        
        y_predictions.append(predicted_label)
        print(f"Test instance {i + 1}\nWeight = {weight}, Height = {height}, Predicted label = {predicted_label}\n")
    
    return y_predictions

In [13]:
def main():
    predictions, accuracy = evaluate_model(X, labels, params)
    print(f"Accuracy on X.csv: {accuracy * 100:.2f}%")
    
    if accuracy == 1.0:
        print("\nPredictions for y.csv:")
        y_predictions = predict_test_instances(y_test, params)
        
        np.savetxt("Q2_Category_Label.csv", y_predictions, fmt="%d")
        print("\nPredictions for y.csv saved to 'Q2_Category_Label.csv'")


if __name__ == "__main__":
    main()

Sample 1
True label = 1, Predicted label = 1, Posterior probs = {1: np.float64(0.999999231612128), 2: np.float64(7.683878719025623e-07), 3: np.float64(3.679673695252389e-25)}

Sample 2
True label = 1, Predicted label = 1, Posterior probs = {1: np.float64(0.9858333595201387), 2: np.float64(0.014166640479860796), 3: np.float64(5.208481360226441e-16)}

Sample 3
True label = 1, Predicted label = 1, Posterior probs = {1: np.float64(0.9999996089783371), 2: np.float64(3.9102166292463367e-07), 3: np.float64(2.20744774981597e-25)}

Sample 4
True label = 1, Predicted label = 1, Posterior probs = {1: np.float64(0.9999997800790268), 2: np.float64(2.1992097321665775e-07), 3: np.float64(1.5826099699707815e-27)}

Sample 5
True label = 1, Predicted label = 1, Posterior probs = {1: np.float64(0.9999997039028208), 2: np.float64(2.9609717923512057e-07), 3: np.float64(1.8230766613548013e-26)}

Sample 6
True label = 1, Predicted label = 1, Posterior probs = {1: np.float64(0.9999985306069831), 2: np.float64